# Imports

In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold



# Read Input Files

In [2]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [3]:
print(train_df.shape, test_df.shape)

(200000, 202) (200000, 201)


In [4]:
train_features = train_df.drop(['target','ID_code'], axis = 1)
train_labels = train_df['target']
test_features = test_df.drop(['ID_code'],axis = 1)

# Gradient Boosted Trees

In [5]:
param = {
        'bagging_freq': 5,
        'bagging_fraction': 0.38,
        'boost_from_average':'false',
        'boost': 'gbdt',
        'feature_fraction': 0.045,
        'learning_rate': 0.01,
        'max_depth': -1,  
        'metric':'auc',
        'min_data_in_leaf': 80,
        'min_sum_hessian_in_leaf': 10.0,
        'num_leaves': 13,
        'num_threads': 4,
        'tree_learner': 'serial',
        'objective': 'binary', 
        'verbosity': 1
    }

In [6]:
num_round = 100000
kfold = 15
folds = StratifiedKFold(n_splits=kfold, shuffle=False, random_state=333)
oof = np.zeros(len(train_df))
predictions_lgb = np.zeros(len(test_df))

In [7]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, train_labels.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_features.iloc[trn_idx], label=train_labels.iloc[trn_idx])
    val_data = lgb.Dataset(train_features.iloc[val_idx], label=train_labels.iloc[val_idx])
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    oof[val_idx] = clf.predict(train_features.iloc[val_idx], num_iteration=clf.best_iteration)
    predictions_lgb += clf.predict(test_features, num_iteration=clf.best_iteration) / folds.n_splits
    
print("CV score: {:<8.5f}".format(roc_auc_score(train_labels, oof)))

Fold 0
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.900513	valid_1's auc: 0.88338
[2000]	training's auc: 0.911164	valid_1's auc: 0.89165
[3000]	training's auc: 0.918219	valid_1's auc: 0.896051
[4000]	training's auc: 0.923726	valid_1's auc: 0.898287
[5000]	training's auc: 0.928367	valid_1's auc: 0.899791
[6000]	training's auc: 0.93243	valid_1's auc: 0.900464
[7000]	training's auc: 0.936127	valid_1's auc: 0.901371
[8000]	training's auc: 0.939649	valid_1's auc: 0.901657
[9000]	training's auc: 0.943064	valid_1's auc: 0.902047
[10000]	training's auc: 0.946264	valid_1's auc: 0.902382
[11000]	training's auc: 0.949375	valid_1's auc: 0.902402
[12000]	training's auc: 0.952391	valid_1's auc: 0.902436
[13000]	training's auc: 0.955217	valid_1's auc: 0.902494
[14000]	training's auc: 0.957985	valid_1's auc: 0.902719
[15000]	training's auc: 0.960678	valid_1's auc: 0.902886
[16000]	training's auc: 0.963208	valid_1's auc: 0.902854
[17000]	training's auc: 0.965

[1000]	training's auc: 0.900034	valid_1's auc: 0.888621
[2000]	training's auc: 0.910996	valid_1's auc: 0.894956
[3000]	training's auc: 0.918157	valid_1's auc: 0.89849
[4000]	training's auc: 0.923677	valid_1's auc: 0.900483
[5000]	training's auc: 0.928371	valid_1's auc: 0.901463
[6000]	training's auc: 0.932469	valid_1's auc: 0.901887
[7000]	training's auc: 0.936197	valid_1's auc: 0.902004
[8000]	training's auc: 0.939625	valid_1's auc: 0.901983
[9000]	training's auc: 0.942996	valid_1's auc: 0.902062
[10000]	training's auc: 0.946206	valid_1's auc: 0.902094
[11000]	training's auc: 0.949366	valid_1's auc: 0.901836
Early stopping, best iteration is:
[8410]	training's auc: 0.94105	valid_1's auc: 0.902148
Fold 10
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.900563	valid_1's auc: 0.878374
[2000]	training's auc: 0.911319	valid_1's auc: 0.887064
[3000]	training's auc: 0.918456	valid_1's auc: 0.891282
[4000]	training's auc: 0.923959	valid_1's auc: 0.8938

In [8]:
submission = pd.DataFrame({"ID_code": test_df.ID_code.values})
submission["target"] = predictions_lgb
submission.to_csv("submission_lgb2.csv", index=False)

# Neural Net

In [10]:
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.optimizer import Optimizer
import time

In [22]:
class Simple_NN(nn.Module):
    def __init__(self ,input_dim ,hidden_dim, dropout = 0.1):
        super(Simple_NN, self).__init__()
        
        self.inpt_dim = input_dim
        self.hidden_dim = hidden_dim
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, int(hidden_dim/2))
        self.fc4 = nn.Linear(int(hidden_dim/2), int(hidden_dim/2))
        self.fc5 = nn.Linear(int(hidden_dim/2), int(hidden_dim/4))
        self.fc6 = nn.Linear(int(hidden_dim/4), int(hidden_dim/8))
        self.fc7 = nn.Linear(int(hidden_dim/8), 1)
#         self.bn1 = nn.BatchNorm1d(hidden_dim)
#         self.bn2 = nn.BatchNorm1d((hidden_dim))
#         self.bn3 = nn.BatchNorm1d(int(hidden_dim/4))
    
    def forward(self, x):
        y = self.fc1(x)
        y = self.relu(y)
        #y = self.bn1(y)
        #y = self.dropout(y)
        
        y = self.fc2(y)
        y = self.relu(y)
        #y = self.bn2(y)
        #y = self.dropout(y)
        
        y = self.fc3(y)
        y = self.relu(y)
        #y = self.bn3(y)
        #y = self.dropout(y)
        
        y = self.fc4(y)
        y = self.relu(y)
        #y = self.bn3(y)
        #y = self.dropout(y)
        
        y = self.fc5(y)
        y = self.relu(y)
        #y = self.bn3(y)
        #y = self.dropout(y)
        
        y = self.fc6(y)
        y = self.relu(y)
        #y = self.bn3(y)
        #y = self.dropout(y)
        
        out= self.fc7(y)
        
        return out

In [23]:
class CyclicLR(object):
    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

In [24]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [25]:
model = Simple_NN(200,1024)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0002)

In [26]:
n_epochs = 40
batch_size = 512

train_preds = np.zeros((len(train_features)))
test_preds = np.zeros((len(test_features)))

x_test = np.array(test_features)
x_test_cuda = torch.tensor(x_test, dtype=torch.float).cuda()
test = torch.utils.data.TensorDataset(x_test_cuda)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

avg_losses_f = []
avg_val_losses_f = []

for i, (train_idx, valid_idx) in enumerate(list(folds.split(train_features, train_labels))):  
    x_train = np.array(train_features)
    y_train = np.array(train_labels)
    
    x_train_fold = torch.tensor(x_train[train_idx.astype(int)], dtype=torch.float).cuda()
    y_train_fold = torch.tensor(y_train[train_idx.astype(int), np.newaxis], dtype=torch.float32).cuda()
    
    x_val_fold = torch.tensor(x_train[valid_idx.astype(int)], dtype=torch.float).cuda()
    y_val_fold = torch.tensor(y_train[valid_idx.astype(int), np.newaxis], dtype=torch.float32).cuda()
    
    loss_fn = torch.nn.BCEWithLogitsLoss()
    
    step_size = 2000
    base_lr, max_lr = 0.0001, 0.005  
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=max_lr)
    
    train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
    valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
    
    ################################################################################################
    scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr,
               step_size=step_size, mode='exp_range',
               gamma=0.99994)
    ###############################################################################################
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
    
    print(f'Fold {i + 1}')
    for epoch in range(n_epochs):
        start_time = time.time()
        model.train()
        avg_loss = 0.
        #avg_auc = 0.
        for i, (x_batch, y_batch) in enumerate(train_loader):
            y_pred = model(x_batch)
            
            #########################
            if scheduler:
                scheduler.batch_step()
            ########################
            
            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            avg_loss += loss.item()/len(train_loader)
            #avg_auc += round(roc_auc_score(y_batch.cpu(),y_pred.detach().cpu()),4) / len(train_loader)
        model.eval()
        
        valid_preds_fold = np.zeros((x_val_fold.size(0)))
        test_preds_fold = np.zeros((len(test_features)))
        
        avg_val_loss = 0.
        #avg_val_auc = 0.
        for i, (x_batch, y_batch) in enumerate(valid_loader):
            y_pred = model(x_batch).detach()
            
            #avg_val_auc += round(roc_auc_score(y_batch.cpu(),sigmoid(y_pred.cpu().numpy())[:, 0]),4) / len(valid_loader)
            avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
            valid_preds_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
            
        elapsed_time = time.time() - start_time 
        print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
            epoch + 1, n_epochs, avg_loss, avg_val_loss, elapsed_time))
        
    avg_losses_f.append(avg_loss)
    avg_val_losses_f.append(avg_val_loss) 
    
    for i, (x_batch,) in enumerate(test_loader):
        y_pred = model(x_batch).detach()

        test_preds_fold[i * batch_size:(i+1) * batch_size] = sigmoid(y_pred.cpu().numpy())[:, 0]
        
    train_preds[valid_idx] = valid_preds_fold
    test_preds += test_preds_fold / folds.n_splits

auc_nn  =  round(roc_auc_score(train_labels,train_preds),4)      
print('All \t loss={:.4f} \t val_loss={:.4f} \t auc={:.4f}'.format(np.average(avg_losses_f),np.average(avg_val_losses_f),auc_nn))

Fold 1
Epoch 1/40 	 loss=0.2834 	 val_loss=0.2940 	 time=7.72s
Epoch 2/40 	 loss=0.2562 	 val_loss=0.3178 	 time=7.62s
Epoch 3/40 	 loss=0.2532 	 val_loss=0.2841 	 time=7.60s
Epoch 4/40 	 loss=0.2581 	 val_loss=0.2962 	 time=7.63s
Epoch 5/40 	 loss=0.2547 	 val_loss=0.2817 	 time=7.66s
Epoch 6/40 	 loss=0.2563 	 val_loss=0.2772 	 time=7.66s
Epoch 7/40 	 loss=0.2490 	 val_loss=0.2932 	 time=7.63s
Epoch 8/40 	 loss=0.2453 	 val_loss=0.2852 	 time=7.69s
Epoch 9/40 	 loss=0.2410 	 val_loss=0.2711 	 time=7.58s
Epoch 10/40 	 loss=0.2376 	 val_loss=0.2771 	 time=7.65s
Epoch 11/40 	 loss=0.2348 	 val_loss=0.2693 	 time=7.64s
Epoch 12/40 	 loss=0.2320 	 val_loss=0.2688 	 time=8.14s
Epoch 13/40 	 loss=0.2343 	 val_loss=0.2708 	 time=7.76s
Epoch 14/40 	 loss=0.2356 	 val_loss=0.2758 	 time=7.73s
Epoch 15/40 	 loss=0.2362 	 val_loss=0.2777 	 time=8.91s
Epoch 16/40 	 loss=0.2385 	 val_loss=0.2698 	 time=8.53s
Epoch 17/40 	 loss=0.2370 	 val_loss=0.2790 	 time=8.47s
Epoch 18/40 	 loss=0.2369 	 val_l

KeyboardInterrupt: 

In [ ]:
submission = pd.DataFrame({"ID_code": test_df.ID_code.values})
submission["target"] = test_preds
submission.to_csv("submission_NN.csv", index=False)

# Ensemble

In [ ]:
ensemble_train_pred = 0.5*oof + 0.5*train_preds
ensemble_test_pred = 0.5*predictions_lgb + 0.5*test_preds

In [ ]:
print("LightGBM AUC: {:<8.5f}".format(roc_auc_score(train_labels, oof)))
print("NN AUC: {:<8.5f}".format(roc_auc_score(train_labels,train_preds)))
print("LightGBM AUC: {:<8.5f}".format(roc_auc_score(train_labels, oof)))

In [ ]:
submission = pd.DataFrame({"ID_code": test_df.ID_code.values})
submission["target"] = ensemble_test_pred
submission.to_csv("submission_ensemble.csv", index=False)